### **1. Importación de librerías**

In [1]:
import tensorflow as tf
from pathlib import Path
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
import os

Se importa las librerías necesarias:

* tensorflow: Principal librería usada para construir y entrenar la CNN.
* Path de pathlib: Sirve para manejar rutas de archivos y directorios de manera flexible.
* Sequential, Conv2D, y MaxPooling2D de tensorflow.keras.layers: Componentes específicos para construir una red CNN secuencial.

### **2. Obtención del directorio de trabajo actual**

In [ ]:
current_directory = Path(os.getcwd())
print(current_directory)

c:\Users\karlo\Desktop\Emotion_recognition\SW_Inteligente_Tarea_4


Se obtiene el directorio de trabajo actual y lo guarda en current_directory. La función print imprime este directorio para confirmar que el script está corriendo en la ruta correcta, lo cual es útil para organizar rutas de archivos en sistemas operativos diferentes.

### **3. Generador de datos de entrenamiento**

In [ ]:
training_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
training_gen = training_data_gen.flow_from_directory(directory=f'{current_directory}/data/train', target_size=(48, 48), color_mode='grayscale', batch_size=32, class_mode='categorical')

Found 28709 images belonging to 7 classes.


Se crea un generador de datos para el conjunto de entrenamiento:

* ImageDataGenerator(rescale=1./255): Normaliza los valores de los píxeles de las imágenes entre 0 y 1 (originalmente están entre 0 y 255).
* flow_from_directory: Carga las imágenes desde el directorio /data/train, con las siguientes configuraciones:
    * target_size=(48, 48): Redimensiona las imágenes a 48x48 píxeles.
    * color_mode='grayscale': Carga las imágenes en escala de grises (1 canal).
    * batch_size=32: Genera los datos en lotes de 32 imágenes.
    * class_mode='categorical': Usa la clasificación categórica para múltiples categorías de emociones.

### **4. Generador de datos de prueba**

In [ ]:
test_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_gen = test_data_gen.flow_from_directory(directory=f'{current_directory}/data/test', target_size=(48, 48), color_mode='grayscale', batch_size=32, class_mode='categorical')

Found 7178 images belonging to 7 classes.


Similar al anterior, pero se configura para el conjunto de prueba en /data/test. Permite evaluar el modelo en datos no vistos, garantizando que el modelo generaliza bien.

### **5. Definición del modelo**

In [ ]:
model = tf.keras.Sequential()
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

Se crea una instancia Sequential, una arquitectura lineal de capas para la CNN. También se define una lista emotions que contiene las categorías de emociones (etiquetas de salida), utilizada más adelante para determinar el tamaño de la capa de salida.

### **6. Capas convolucionales y de agrupamiento**

In [ ]:
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))

c:\Users\karlo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Se agrega capas de convolución y agrupamiento:

* Primera capa: Conv2D con 32 filtros y tamaño de kernel 3x3, relu como función de activación y input_shape=(48, 48, 1) (48x48 píxeles y 1 canal).
* Primera capa MaxPooling: Reduce la dimensión de la imagen a la mitad usando una ventana de 2x2.
* Segunda capa Conv2D: Conv2D con 64 filtros y un tamaño de kernel de 3x3, seguido de otra capa MaxPooling.
* Tercera capa Conv2D: Nueva capa Conv2D de 64 filtros.

Estas capas ayudan a extraer características de las imágenes, como bordes y texturas.

### **7. Aplanamiento de las salidas**

In [ ]:
model.add(tf.keras.layers.Flatten())

Convierte la salida tridimensional de las capas convolucionales en una sola dimensión, preparándola para las capas densas que seguirán.

### **8. Capas densas para clasificación**

In [ ]:
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(len(emotions), activation='softmax'))

Se agrega capas densas para la clasificación:

* La primera capa densa tiene 64 unidades y relu como función de activación.
* La segunda capa densa (capa de salida) tiene tantas unidades como categorías de emociones (len(emotions)), con activación softmax para producir una distribución de probabilidad para cada clase.

### **9. Compilación del modelo**

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Se compila el modelo con:

* optimizer='adam': Optimizador Adam para ajustar los pesos del modelo.
* loss='categorical_crossentropy': Pérdida adecuada para clasificación multiclase.
* metrics=['accuracy']: Métrica de precisión para monitorear el rendimiento.

### **10. Entrenamiento del modelo**

In [ ]:
history = model.fit(training_gen, epochs=20, validation_data=test_gen)

Epoch 1/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.4682 - loss: 1.3877 - val_accuracy: 0.4702 - val_loss: 1.3883
Epoch 2/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.5244 - loss: 1.2540 - val_accuracy: 0.5244 - val_loss: 1.2511
Epoch 3/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - accuracy: 0.5719 - loss: 1.1417 - val_accuracy: 0.5293 - val_loss: 1.2327
Epoch 4/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.6003 - loss: 1.0661 - val_accuracy: 0.5435 - val_loss: 1.2125
Epoch 5/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.6333 - loss: 0.9841 - val_accuracy: 0.5578 - val_loss: 1.1955
Epoch 6/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.6687 - loss: 0.9002 - val_accuracy: 0.5396 - val_loss: 1.2579
Epoch 7/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 18s 20ms/step - accuracy: 0.6964 - loss: 0.8222 - val_accuracy: 0.5414 - val_loss: 1.2979
Epoch 8/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 18s 20ms/step - accuracy: 0.7311 - loss: 0.7417 - 

Este bloque entrena el modelo:

* training_gen: Datos de entrenamiento.
* epochs=20: Número de épocas para entrenar.
* validation_data=test_gen: Datos de validación para evaluar el modelo al final de cada época.

### **11. Guardado del modelo entrenado**

In [ ]:
model.save(f'{current_directory}/model/trained_model.h5')

Finalmente, el modelo entrenado se guarda en un archivo .h5 en el directorio /model/. Esto permite reutilizar el modelo sin necesidad de volver a entrenarlo.